# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

# 1. Importando Bibliotecas e Banco de Dados

In [62]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [63]:
sinasc.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


# 2. Tratamento dos Dados

In [64]:
# 2) seu código aqui
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [65]:
# 3) seu código aqui
sinasc_trat = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO','GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] ]
sinasc_trat.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0


In [66]:
sinasc_trat.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [67]:
# 4) seu código aqui
sinasc_trat.dropna(subset = ['APGAR5'], inplace = True)
sinasc_trat.isna().sum()

<ipython-input-67-bb228ba371ce>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_trat.dropna(subset = ['APGAR5'], inplace = True)


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [68]:
sinasc_trat.shape

(26925, 9)

In [69]:
# 5) seu código aqui
cod = sinasc_trat[(sinasc_trat['ESTCIVMAE'] == 9) | (sinasc_trat['CONSULTAS'] == 9)].index
cod

Int64Index([    2,  2867,  3637,  3713,  3735,  4108,  4290,  4392,  4406,
             4459,
            ...
            25694, 26095, 26212, 26486, 26528, 26542, 26548, 26759, 26791,
            26973],
           dtype='int64', length=194)

In [70]:
print(sinasc_trat['CONSULTAS'].value_counts())
print(sinasc_trat['ESTCIVMAE'].value_counts())

4    18587
3     5957
2     1359
1      972
9       50
Name: CONSULTAS, dtype: int64
2.0    9906
1.0    8907
5.0    7145
4.0     451
9.0     144
3.0      57
Name: ESTCIVMAE, dtype: int64


In [71]:
sinasc_drop = sinasc_trat.drop(cod)
sinasc_drop.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0


**Verificação** 

In [72]:
print(sinasc_drop['CONSULTAS'].value_counts())
print(sinasc_drop['ESTCIVMAE'].value_counts())

4    18486
3     5925
2     1354
1      966
Name: CONSULTAS, dtype: int64
2.0    9893
1.0    8881
5.0    7135
4.0     450
3.0      57
Name: ESTCIVMAE, dtype: int64


In [73]:
# 6) Seu código aqui
sinasc_drop['QTDFILVIVO'].fillna(0, inplace = True)
sinasc_drop

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [74]:
sinasc_drop.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         288
QTDFILVIVO       0
GESTACAO      1197
GRAVIDEZ        75
CONSULTAS        0
APGAR5           0
dtype: int64

In [75]:
sinasc_drop.dtypes

LOCNASC         int64
IDADEMAE        int64
ESTCIVMAE     float64
ESCMAE         object
QTDFILVIVO    float64
GESTACAO       object
GRAVIDEZ       object
CONSULTAS       int64
APGAR5        float64
dtype: object

Para entender melhor qual valor subtituir o restante das colunas que possuem NaN, é interessante compreender melhor cada coluna.
Nas que são tipo númerico (int ou float), seria interessante pegar a média ou mediana, já do tipo string (ou object), o valor mais comum, se for um valor muito mais numeroso em relação aos outros.

In [86]:
# 7) seu código aqui
print(sinasc_drop['ESTCIVMAE'].value_counts())

2.0    9893
1.0    8881
5.0    7135
4.0     450
3.0      57
Name: ESTCIVMAE, dtype: int64


Pela coluna ESTCIVMAE apresentar valores inteiros, onde cada valor tem um significado, a melhor opção é uitlizar a mediana para substituir os valores NaN

In [87]:
mediana_est = sinasc_drop['ESTCIVMAE'].median()
mediana_est

2.0

In [88]:
sinasc_drop['ESTCIVMAE'].fillna(mediana_est, inplace = True)
sinasc_drop.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         288
QTDFILVIVO       0
GESTACAO      1197
GRAVIDEZ        75
CONSULTAS        0
APGAR5           0
dtype: int64

In [89]:
print(sinasc_drop['ESCMAE'].value_counts(1))

8 a 11 anos        0.585297
12 anos ou mais    0.216466
4 a 7 anos         0.177552
1 a 3 anos         0.019211
Nenhuma            0.001475
Name: ESCMAE, dtype: float64


Para as colunas ESCMAE, GESTACAO e GRAVIDEZ achei mais interessante colocar o valor que aparece com maior frequência

In [90]:
sinasc_drop['ESCMAE'].fillna('8 a 11 anos', inplace = True)
sinasc_drop.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE           0
QTDFILVIVO       0
GESTACAO      1197
GRAVIDEZ        75
CONSULTAS        0
APGAR5           0
dtype: int64

In [91]:
print(sinasc_drop['GESTACAO'].value_counts(1))

37 a 41 semanas        0.875969
32 a 36 semanas        0.087021
42 semanas e mais      0.027140
28 a 31 semanas        0.006305
22 a 27 semanas        0.003290
Menos de 22 semanas    0.000274
Name: GESTACAO, dtype: float64


In [92]:
sinasc_drop['GESTACAO'].fillna('37 a 41 semanas', inplace = True)
sinasc_drop.isna().sum()

LOCNASC        0
IDADEMAE       0
ESTCIVMAE      0
ESCMAE         0
QTDFILVIVO     0
GESTACAO       0
GRAVIDEZ      75
CONSULTAS      0
APGAR5         0
dtype: int64

In [93]:
print(sinasc_drop['GRAVIDEZ'].value_counts(1))

Única              0.981543
Dupla              0.018270
Tríplice e mais    0.000188
Name: GRAVIDEZ, dtype: float64


In [94]:
sinasc_drop['GRAVIDEZ'].fillna('Única', inplace = True)
sinasc_drop.isna().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

O Apgar possui uma classificação indicando se o bebê passou por asfixia:

Entre 8 e 10 está em uma faixa 'normal'.
Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
Entre 4 e 5 significa 'asfixia moderada'.
Entre 0 e 3 significa 'asfixia severa'.

In [97]:
# 8) seu código aqui
sinasc_drop.loc[sinasc_drop['APGAR5'] <= 3, 'APGAR_CAT'] = 'asfixia severa'
sinasc_drop.loc[(sinasc_drop['APGAR5'] == 5) | (sinasc_drop['APGAR5'] == 4), 'APGAR_CAT'] =  'asfixia moderada'
sinasc_drop.loc[(sinasc_drop['APGAR5'] == 6) | (sinasc_drop['APGAR5'] == 7), 'APGAR_CAT'] =  'asfixia leve'
sinasc_drop.loc[sinasc_drop['APGAR5'] >= 8, 'APGAR_CAT'] = 'normal'
sinasc_drop.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR_CAT
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,normal


Renomear as colunas para ficar com letras minusculas

In [99]:
# 9) seu código aqui
sinasc_drop.columns = sinasc_drop.columns.str.lower()
sinasc_drop

,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,apgar_cat
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal
